In [53]:
import geopandas as gpd
import pandas as pd
import wget
import os
import glob

In [2]:
gdf_distritos = gpd.read_file('dados/SIRGAS_GPKG_distrito.gpkg')

In [4]:
gdf_distritos

,ds_areamt,ds_subpref,ds_sigla,ds_nome,ds_areakm,ds_cd_sub,ds_codigo,geometry
0,None,SANTANA-TUCURUVI,MAN,MANDAQUI,None,05,51,"POLYGON ((330950.373 7407837.176, 330952.177 7..."
1,None,MOOCA,MOO,MOOCA,None,25,53,"POLYGON ((338349.356 7394264.944, 338362.239 7..."
2,None,CASA VERDE-CACHOEIRINHA,LIM,LIMAO,None,04,50,"POLYGON ((329079.913 7402343.759, 329081.724 7..."
3,None,JABAQUARA,JAB,JABAQUARA,None,15,38,"POLYGON ((331924.648 7385714.896, 331932.746 7..."
4,None,CIDADE ADEMAR,CAD,CIDADE ADEMAR,None,16,22,"POLYGON ((332410.602 7378610.746, 332404.612 7..."
...,...,...,...,...,...,...,...,...
91,None,GUAIANASES,LAJ,LAJEADO,None,28,96,"POLYGON ((357579.815 7394795.986, 357572.551 7..."
92,None,LAPA,LAP,LAPA,None,08,48,"POLYGON ((323721.814 7398433.381, 323730.119 7..."
93,None,SE,LIB,LIBERDADE,None,09,49,"POLYGON ((332502.702 7392061.729, 332502.811 7..."
94,None,LAPA,JRE,JAGUARE,None,08,41,"POLYGON ((322952.516 7394978.618, 323250.620 7..."


In [34]:
for distrito in gdf_distritos.iterrows():

    url = f'https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_{distrito[1].ds_codigo.ljust(2, "0")}_{distrito[1].ds_nome.replace(" ", "_")}.gpkg'
    file = f'2022/SIRGAS_IPTU_2022_{distrito[1].ds_codigo.ljust(2, "0")}_{distrito[1].ds_nome.replace(" ", "_")}.gpkg'
    print(url)
    
    if not os.path.exists(file):
        _ = wget.download(url, file)

https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_51_MANDAQUI.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_53_MOOCA.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_50_LIMAO.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_38_JABAQUARA.gpkg


URLError: <urlopen error [Errno 110] Connection timed out>

In [43]:
tipos_de_uso = [
    "Loja",
    "Loja e residência (predominância comercial)",
    "Outras edificações de uso comercial, com utilização múltipla",
    "Loja em edifício em condomínio (unidade autônoma)",
    "Armazéns gerais e depósitos",
    "Outras edificações de uso especial, com utilização múltipla"
]

In [55]:
distritos = []

for file in glob.glob('2022/*'):
    gdf_distrito = gpd.read_file(file)
    filtro_usos_comerciais = gdf_distrito.loc[:, 'TIPO DE USO DO IMOVEL'].isin(tipos_de_uso)
    gdf_distrito = gdf_distrito.loc[filtro_usos_comerciais, :].reset_index()
    distritos.append(gdf_distrito)

In [58]:
gdf_estabelecimentos_comerciais = pd.concat(distritos)

In [61]:
gdf_hexagonos = gpd.read_file('dados/Sao-Paulo-hexagonos-250.gpkg')

In [65]:
gdf_esta_hexagon = gdf_estabelecimentos_comerciais.sjoin(gdf_hexagonos)

In [68]:
# gdf_esta_hexagon.columns

In [69]:
atributos = [
    'index_right',
    'AREA CONSTRUIDA'
]

In [75]:
def median(x):
    return x.quantile(0.5)

def q25(x):
    return x.quantile(0.25)

def q75(x):
    return x.quantile(0.75)

In [76]:
gdf_esta_hexagon.loc[:, atributos].groupby(by='index_right').agg(
    {'AREA CONSTRUIDA': ['min', 'max', 'mean', 'std', 'var', median, q25, q75]}
)

AREA CONSTRUIDA                                                 \
                        min    max         mean          std           var   
index_right                                                                  
18915                   202   5750  2054.333333  3200.544381  1.024348e+07   
18916                   647  11232  5939.500000  7484.725279  5.602111e+07   
19200                    62    102    82.000000    28.284271  8.000000e+02   
19203                   100    100   100.000000     0.000000  0.000000e+00   
19204                    71   5322   600.954545  1099.089904  1.207999e+06   
...                     ...    ...          ...          ...           ...   
33443                   150   2091   430.090909   561.564503  3.153547e+05   
33444                   170    820   467.000000   328.598539  1.079770e+05   
33445                   125    891   424.000000   352.515721  1.242673e+05   
33734                   121    323   187.333333    63.912049  4.084750e+03   
33735                    68    531   282.166667   140.736913  1.980688e+04   

                                       
             median      q25      q75  
index_right                            
18915         211.0   206.50  2980.50  
18916        5939.5  3293.25  8585.75  
19200          82.0    72.00    92.00  
19203         100.0   100.00   100.00  
19204         307.0   150.00   509.75  
...             ...      ...      ...  
33443         275.0   174.00   348.50  
33444         411.0   290.50   615.50  
33445         340.0   166.25   597.75  
33734         190.0   139.00   221.00  
33735         269.5   193.75   337.00  

[400 rows x 8 columns]

In [60]:
gdf_estabelecimentos_comerciais.crs

<Derived Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich